# Scrapyd管理部署Scrapy爬虫详解
　　参考：[官方文档](https://scrapyd.readthedocs.io/en/stable/api.html)<br>
　　　　　[配置文档](https://scrapyd.readthedocs.io/en/latest/config.html)<br>
　　　　　[GitHub](https://github.com/scrapy/scrapyd)<br>
　　　　　[Scrapyd使用详解](https://www.jianshu.com/p/9a4ce1e2fa7b)<br>
　　　　　[崔庆才—Scrapyd 的安装](https://cuiqingcai.com/5445.html)<br>
　　　　　[崔庆才—Scrapyd-Client 的使用](https://cuiqingcai.com/8491.html)<br>
　　　　　[scrapyd部署_第八章 第一节 scrapyd和scrapy-client](https://blog.csdn.net/weixin_39622655/article/details/112222306)<br>
　　　　　[基于 Docker 的 Scrapyd 服务部署](https://zhuanlan.zhihu.com/p/160185662)<br>
　　　　　[scrapy+scrapyd+scrapydweb（centos7）部署爬虫（绝世好文）](https://blog.csdn.net/baidu_40492134/article/details/107617338)<br>
　　　　　[基于Docker的Scrapy+Scrapyd+Scrapydweb部署](https://blog.csdn.net/weixin_33696822/article/details/88676398)<br>
　　　　　[scrapy部署 与 发布到scrapyd](https://www.cnblogs.com/angdh/p/11886519.html)<br>
　　　　　[scrapy爬虫docker部署](https://zhuanlan.zhihu.com/p/25562941)<br>
　　　　　[scrapy部署scrapyd+scrapydweb+nginx设置密码+docker](https://www.cnblogs.com/angdh/p/12571379.html)<br>
　　　　　[[Docker]Docker部署Scrapy-redis分布式爬虫框架实践（整合Selenium）](https://zhuanlan.zhihu.com/p/54383612)<br>
　　　　　[]()<br>
　　　　　[]()<br>
　　　　　[]()<br>
　　　　　[]()<br>

## 一、Scrapyd介绍
　　scrapyd 是由scrapy 官方提供的爬虫管理工具（需要单独安装），使用它我们可以非常方便地上传、控制爬虫并且查看运行日志。<br>
　　使用scrapyd 和我们直接运行scrapy crawl myspider有什么区别呢？<br>
　　scrapyd 同样是通过上面的命令运行爬虫的，不同的是它提供一个JSON web service 监听的请求，我们可以从任何一台可以连接到服务器的电脑发送请求安排爬虫运行，或者停止正在运行的爬虫。甚至，我们可以使用它提供的API上传新爬虫而不必登录到服务器上进行操作。<br>
　　Scrapyd通常作为守护进程运行，它侦听运行爬虫的请求，并为每个请求生成一个进程，该进程基本上执行:scrapy crawl [myspider]。<br>
　　Scrapyd还并行运行多个进程，将它们分配到max_proc和max_proc_per_cpu选项提供的固定数量的插槽中，启动尽可能多的进程来处理负载。<br>
　　Scrapyd的web界面比较简单，主要用于监控，所有的调度工作全部依靠接口实现.
## 二、Scrapyd安装配置
### 1、安装
>pip install scrapyd<br>
### 2、修改默认配置信息
#### 2.1、创建配置文件
　　安装完毕之后，需要新建一个配置文件 /etc/scrapyd/scrapyd.conf，Scrapyd 在运行的时候会读取此配置文件。<br>
　　在 Scrapyd 1.2 版本之后，不会自动创建该文件，需要我们自行添加。<br>
　　首先，执行如下命令新建文件：<br>
>sudo mkdir /etc/scrapyd<br>
sudo vi /etc/scrapyd/scrapyd.conf<br>

#### 2.2、修改配置文件
　　官方详细配置文档说明：https://scrapyd.readthedocs.io/en/latest/config.html<br>
　　可以在项目下新建一个scrapyd.conf或者在scrapy.cfg中增加[scrapyd]：<br>
>[scrapyd]              
bind_address = 127.0.0.1　　　# 网页和Json服务监听的IP地址，默认为127.0.0.1<br>
http_port = 6800　　　　　　　# 监听的端口，默认为6800<br>
debug  = off　　　　　　　　　# 是否打开debug模式，默认为off<br>
max_proc_per_cpu = 4　　　　　# 每个CPU最多可启用的Scrapy 进程数，默认为4<br>
max_proc = 0　　　　　　　　　# 可启用的最多进程数，默认为0.如果未设置或者设为0，则使用的最多进程数=CPU数量*max_proc_per_cpu<br>
eggs_dir = eggs　　　　　　　# 项目eggs生成目录，默认为项目目录下eggs<br>
logs_dir = logs　　　　　　　　# 项目日志生成目录，默认为项目目录下logs，如果不想要生成日志，可以直接设置成空<br>
dbs_dir = dbs　　　　　　　　# 项目dbs生成目录，默认为项目目录下dbs<br>
items_dir = 　　　　　　　　# 爬取的items存储的文件夹（版本0.15.以上），默认为空，不存储。<br>
jobs_to_keep = 5　　　　　　　# 每个爬虫保持的完成任务数，默认为5.（版本0.15.以上，以前版本中为logs_to_keep）<br>
finished_to_keep = 100　　　　# 保持的完成任务进程数。默认为100.（版本0.14.以上）<br>
poll_interval = 5.0　　　　　# 轮训请求队列的时间间隔。默认为5s，可以为浮点数<br>
runner = scrapyd.runner   　　# 启动子进程的模块。可以使用自定义 <br>
application = scrapyd.app.application　　　　# 返回可用于twisted的application，可继承于Scrapyd添加和移除自己的组件和服务。https://twistedmatrix.com/documents/current/core/howto/application.html查看更多<br>
launcher = scrapyd.launcher.Launcher    # twisted的web资源，表示到scrapyd的接口。Scrapyd包含一个带有网站的界面，可以提供对应用程序的web资源的简单监视和访问。此设置必须提供twisted web资源的根类。<br>
webroot  = scrapyd.website.Root<br>
<br>
[services]<br>
schedule.json     = scrapyd.webservice.Schedule<br>
cancel.json       = scrapyd.webservice.Cancel<br>
addversion.json   = scrapyd.webservice.AddVersion<br>
listprojects.json = scrapyd.webservice.ListProjects<br>
listversions.json = scrapyd.webservice.ListVersions<br>
listspiders.json  = scrapyd.webservice.ListSpiders<br>
delproject.json   = scrapyd.webservice.DeleteProject<br>
delversion.json   = scrapyd.webservice.DeleteVersion<br>
listjobs.json     = scrapyd.webservice.ListJobs<br>
daemonstatus.json = scrapyd.webservice.DaemonStatus<br>

　　默认情况下scrapyd 监听本地 127.0.0.1:6800端口，在此修改为 0.0.0.0，以使外网可以访问。运行 scrapyd 后在浏览器(本地)http://localhost:6800/ 即可查看到当前可以运行的项目。<br>
　　web接口：<br>
>http://localhost:6800/

#### 2.3、开放6800端口
>firewall-cmd --zone=public --add-port=6800/tcp --permanent　　　　# 开放6800端口<br>
firewall-cmd --zone=public --remove-port=6800/tcp --permanent　　　#关闭6800端口<br>
firewall-cmd --reload　　　　　　　　　　　　　　　　　　　　　　　# 配置立即生效<br>
firewall-cmd --zone=public --list-ports　　　　　　　　　　　# 查看防火墙所有开放的端口<br>
netstat -lnpt　　　　　　　　　　　　　　　　　　　　　　　　　　　# 查看监听的端口<br>
systemctl stop firewalld.service　　# 如果要开放的端口太多，可以关闭防火墙，安全性自行评估<br>
firewall-cmd --state　　　　　　# 查看防火墙状态<br>

### 3、运行Scrapyd服务
#### 3.1、直接运行crapyd
　　Scrapyd 是一个纯 Python 项目，这里可以直接调用它来运行。<br>
>scrapyd<br>
#### 3.2、后台运行crapyd
　　为了使程序一直在后台运行，Linux 和 Mac 可以使用如下命令：<br>
>(scrapyd > /dev/null \&)<br>

　　这样 Scrapyd 就会在后台持续运行了，控制台输出直接忽略。<br>
#### 3.3、直接运行crapyd
　　如果想记录输出日志，可以修改输出目标，如：<br>
>(scrapyd > ~/scrapyd.log &)<br>

　　此时会将 Scrapyd 的运行结果输出到～/scrapyd.log 文件中。<br>
<br>
　　当然也可以使用 screen、tmux、supervisor 等工具来实现进程守护（运行 Scrapyd 更佳的方式是使用 Supervisor 守护进程，如果感兴趣，可以参考:http://supervisord.org/ 。<br>
　　运行之后，便可以在浏览器的 6800 端口访问 Web UI 了，从中可以看到当前 Scrapyd 的运行任务、日志等内容。<br>

### 4、访问认证
　　配置完成后，Scrapyd 和它的接口都是可以公开访问的。如果想配置访问认证的话，可以借助于 Nginx 做反向代理，这里需要先安装 Nginx 服务器。<br>
　　以 Ubuntu 为例进行说明，安装命令如下：<br>
>sudo apt-get install nginx<br>

　　然后修改 Nginx 的配置文件 nginx.conf，增加如下配置：<br>
>http {<br>
　　server {<br>
　　　　listen 6801;<br>
　　　　location / {<br>
　　　　　　proxy_pass    http://127.0.0.1:6800/;<br>
　　　　　　auth_basic    "Restricted";<br>
　　　　　　auth_basic_user_file    /etc/nginx/conf.d/.htpasswd;<br>
　　　　}<br>
　　}<br>
}<br>

　　这里使用的用户名和密码配置放置在 /etc/nginx/conf.d 目录下，我们需要使用 htpasswd 命令创建。例如，创建一个用户名为 admin 的文件，命令如下：<br>
>htpasswd -c .htpasswd admin

　　接着就会提示我们输入密码，输入两次之后，就会生成密码文件。此时查看这个文件的内容：<br>
>cat .htpasswd <br>
admin:5ZBxQr0rCqwbc<br>

　　配置完成后，重启一下 Nginx 服务，运行如下命令：<br>
>sudo nginx -s reload

　　这样就成功配置了 Scrapyd 的访问认证了。<br>

## 三、 Scrapyd-Client
　　Scrapy项目部署有现成的工具来完成，它叫作 Scrapyd-Client。
### 3.1、Scrapyd-Client安装与验证
>pip3 install scrapyd-client<br>
scrapyd-deploy -h<br>

　　安装成功后会有一个可用命令，叫做 scrapyd-deploy，即为部署命令。我们利用它来测试 Scrapyd-Client 是否安装成功。<br>

### 3.2、Scrapyd-Client的功能
　　**Scrapyd部署主要分为两步：**
>1、将项目打包成egg文件。<br>
2、将打包生成的egg文件通过Scrapyd的addversion.json接口上传到目标服务器的Scrapyd上。<br>

　　Scrapyd-Client 帮我们把部署全部实现了，我们不需要再去关心 Egg 文件是怎样生成的，也不需要再去读 Egg 文件并请求接口上传了，这一切的操作只需要执行一个命令即可一键部署。<br>

### 3.3、 Scrapyd-Client 部署
　　要部署 Scrapy 项目，我们首先需要修改一下项目的配置文件，例如一个 Scrapy 微博爬虫项目，在项目的第一层会有一个 scrapy.cfg 文件，它的内容如下：
>[settings]<br>
default = weibo.settings<br>
<br>
[deploy]<br>
#url = http://localhost:6800/<br>
project = weibo<br>

　　修改工程目录下的 scrapy.cfg 文件：<br>
>[deploy:scrapyd2]　　　　　# 如果我们有多台主机，我们可以配置多个 deploy 名称来，分别配置各台主机的url<br>
url = http://scrapyd.mydomain.com/api/scrapyd/　　　  # 要部署项目的服务器的地址<br>
username = john 　　　　　　　　　　 # 访问服务器所需的用户名和密码（如果不需要密码可以不写）<br>
password = secret<br>

　　其中的username 和 password 用于在部署时验证服务器的HTTP basic authentication，须要注意的是这里的用户密码并不表示访问该项目须要验证，而是登录服务器用的。<br>
　　Ubuntu/Windows:
>[deploy:tutorial_deploy]<br>
url = http://192.168.17.129:6800/ 　　　  # 要部署项目到 192.168.17.129 的 Scrapyd 上<br>
project = tutorial<br>
username = enlong<br>
password = test<br>


　　**部署项目到服务器**,直接在项目根目录(即scrapy.cfg所在的目录下运行:scrapyd-deploy)：
　　Windows:
>python c:\Python27\Scripts\scrapyd-deploy

　　Ubuntu:
>scrapyd-deploy tutorial_deploy -p tutorial

　　部署操作会打包你的当前项目，如果当前项目下有setup.py文件，就会使用它，没有的会就会自动创建一个(如果后期项目需要打包的话，可以根据自己的需要修改里面的信息，也可以暂时不管它)。从返回的结果里面，可以看到部署的状态，项目名称，版本号和爬虫个数，以及当前的主机名称。<br>
　　注意：部署前，需要先在项目目录下运行scrapyd，启动Scrapyd服务。
  
### 3.4、 多台主机部署
　　另外如果我们有多台主机，我们可以配置各台主机的别名，例如可以修改配置文件scrapy.cfg为：
>[deploy:vm1]<br>
url = http://120.27.34.24:6800/<br>
project = weibo<br>
<br>
[deploy:vm2]<br>
url = http://139.217.26.30:6800/<br>
project = weibo<br>

　　有多台主机的话就在此统一配置，一台主机对应一组配置，在 deploy 后面加上主机的别名即可，这样如果我们想将项目部署到 IP 为 139.217.26.30 的 vm2 主机，我们只需要执行如下命令：
>scrapyd-deploy vm2

　　这样我们就可以将项目部署到名称为 vm2 的主机上了。 如此一来，如果我们有多台主机，我们只需要在 scrapy.cfg 文件中配置好各台主机的 Scrapyd 地址，然后调用 scrapyd-deploy 命令加主机名称即可实现部署，非常方便。 如果 Scrapyd 设置了访问限制的话，我们可以在配置文件中加入用户名和密码的配置，同时端口修改一下，修改成 Nginx 代理端口，如在第一章我们使用的是 6801，那么这里就需要改成 6801，修改如下：
>[deploy:vm1]<br>
url = http://120.27.34.24:6801/<br>
project = weibo<br>
username = admin<br>
password = admin<br>
<br>
[deploy:vm2]<br>
url = http://139.217.26.30:6801/<br>
project = weibo<br>
username = germey<br>
password = germey<br>

　　这样通过加入 username 和 password 字段我们就可以在部署时自动进行 Auth 验证，然后成功实现部署。<br>

## 四、Scrapyd使用
### 4.1、查看项目spider
　　通过scrapyd-deploy -l 查看当前目录下的可以使用的部署方式（target）<br>
　　Windows/Ubuntu:
>scrapy list<br>
scrapyd-deploy -l

　　或再次打开http://localhost:6800/, 也可以看到Available projects: default, tutorial。列出服务器上所有的项目，检查tutorial_deploy是否已经部署上去了:
>scrapyd-deploy -L tutorial_deploy<br>
default<br>
tutorial

### 4.2、开启爬虫 schedule
>curl http://localhost:6800/schedule.json -d project=tutorial -d spider=tencent

　　Windows/Ubuntu 注意：执行时 cd 到项目根目录执行
>curl http://localhost:6800/schedule.json -d project=tutorial -d spider=tencent<br>
{"status": "ok", "jobid": "94bd8ce041fd11e6af1a000c2969bafd", "node_name": "ubuntu"}　　　　# 返回状态值

### 4.3、停止 cancel
>curl http://localhost:6800/cancel.json -d project=tutorial -d job=94bd8ce041fd11e6af1a000c2969bafd

### 4.4、列出爬虫
>curl http://localhost:6800/listspiders.json?project=tutorial

### 4.5、删除项目
>curl http://localhost:6800/delproject.json -d project=tutorial

### 4.6、更新
　　对于Scrapyd默认项目(即是启动Scrapyd命令后看到的default项目)：<br>
　　只有在scrapy项目里启动scrapyd命令时才有默认项目，默认项目就是当前的scrapy项.如果在非scrapy项目下执行scrapyd, 是看不到default的<br>
　　注意：执行时 cd 到项目根目录执行<br>
　　第一种情况cfg:
>[deploy]<br>
url = http://192.168.17.129:6800/<br>
project = tutorial<br>
username = enlong<br>
password = test

　　运行结果：
>python@ubuntu:~/project/tutorial$ scrapyd-deploy <br>
Packing version 1471069533<br>
Deploying to project "tutorial" in http://192.168.17.129:6800/addversion.json<br>
Server response (200):<br>
{"status": "ok", "project": "tutorial", "version": "1471069533", "spiders": 1, "node_name": "ubuntu"}<br>

　　第二种情况cfg:
>[deploy:tutorial_deploy]<br>
url = http://192.168.17.129:6800/<br>
project = tutorial<br>
username = enlong<br>
password = test

　　运行结果：
>python@ubuntu:~/project/tutorial$ scrapyd-deploy tutorial_deploy<br>
Packing version 1471069591<br>
Deploying to project "tutorial" in http://192.168.17.129:6800/addversion.json<br>
Server response (200):<br>
{"status": "ok", "jobid": "94bd8ce041fd11e6af1a000c2969bafd", "node_name": "ubuntu"}<br>

